In [186]:
import numpy as np
import utils.porter as pt
import Parser as parse
import TextRepresenter as tr
import collections
import Index as ind
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [131]:
Documents = ['the new home has been saled on top forecasts','the home sales rise in july', 'there is an increase in home sales in july',' july encounter a new home sales rise']

In [206]:
parser = parse.Parser()
parsed = parser.buildDocCollectionSimple("../data/cisi/cisi.txt",balise='.W')
index = ind.IndexerSimple(parser.collection)

In [207]:
len(index.index)

2459

In [217]:
def testLong():

    
    parsed = None
    file = "data/cacm/cacm.txt"
    for path in ["./", "../"]:
        try:
            parser = parse.Parser()
            _ = parser.buildDocCollectionSimple(path + file)
            break
        except FileNotFoundError:
            pass
    
    print("création index")
    indexer = ind.IndexerSimple(parser.collection)
    print(len(indexer.index))
    # assert len(indexer.index) == 2459
    
    print("création tfidf")
    tf_idf = indexer.tf_idf
    assert tf_idf.keys() == indexer.index.keys()

In [218]:
testLong()

création index
4202
création tfidf


In [215]:
testShort()

In [135]:
index.get_normalized_index()[1]

{'report-intern': {'1': 0.2},
 'languag': {'1': 0.2},
 'techniqu': {'3': 0.2},
 'transform': {'8': 0.25},
 'machin': {'11': 0.125},
 'extract': {'2': 0.16666666666666666},
 'depart': {'3': 0.2},
 'propos': {'9': 0.5, '11': 0.125},
 'terminolog': {'4': 0.2, '7': 0.2, '10': 0.2},
 'engin': {'4': 0.2, '7': 0.2, '10': 0.2},
 'digit': {'2': 0.16666666666666666},
 'programm': {'4': 0.2, '7': 0.2, '10': 0.2, '11': 0.125},
 'procedur': {'6': 0.3333333333333333},
 'inspect': {'6': 0.3333333333333333},
 'equival': {'8': 0.25},
 'chang': {'11': 0.125},
 'glossari': {'4': 0.2, '7': 0.2, '10': 0.2},
 'uncol': {'9': 0.5},
 'solution-part': {'11': 0.125},
 'problem': {'11': 0.125},
 'root': {'2': 0.16666666666666666},
 'preliminari': {'1': 0.2},
 'repeat': {'2': 0.16666666666666666},
 'scheme': {'3': 0.2, '8': 0.25},
 'matrix': {'3': 0.2},
 'program': {'3': 0.2, '8': 0.25},
 '2': {'11': 0.125},
 'comput': {'2': 0.16666666666666666,
  '4': 0.2,
  '6': 0.3333333333333333,
  '7': 0.2,
  '10': 0.2},
 'ap

In [48]:
print(index.index)

{'1': {'preliminari': 1, 'report-intern': 1, 'algebra': 1, 'languag': 1, 'jgkfldjgfkld': 1}, '2': {'extract': 1, 'root': 1, 'repeat': 1, 'subtract': 1, 'digit': 1, 'comput': 1}, '3': {'techniqu': 1, 'depart': 1, 'matrix': 1, 'program': 1, 'scheme': 1}, '4': {'glossari': 1, 'comput': 1, 'engin': 1, 'programm': 1, 'terminolog': 1}, '5': {'square-root': 1, 'approxim': 1}, '6': {'comput': 1, 'inspect': 1, 'procedur': 1}, '7': {'glossari': 1, 'comput': 1, 'engin': 1, 'programm': 1, 'terminolog': 1}, '8': {'equival': 1, 'transform': 1, 'program': 1, 'scheme': 1}, '9': {'propos': 1, 'uncol': 1}, '10': {'glossari': 1, 'comput': 1, 'engin': 1, 'programm': 1, 'terminolog': 1}, '11': {'problem': 1, 'programm': 1, 'commun': 1, 'chang': 1, 'machin': 1, 'propos': 1, 'solution-part': 1, '2': 1}}


In [70]:
index.create_tf_idf()

AttributeError: 'IndexerSimple' object has no attribute 'create_tf_idf'